In [ ]:
import pandas as pd

In [12]:
import glob
from zipfile import ZipFile

def init_dataframe_from_zip(path, index_col):
    df_master = pd.DataFrame()
    flag = False
    files_in_folder = glob.glob(path)
    for filename in files_in_folder:    
        zip_file = ZipFile(filename)
        for text_file in zip_file.infolist():
            if text_file.filename.endswith('.csv'):
                df = pd.read_csv(zip_file.open(text_file.filename),header=0,index_col=[index_col])
                if not flag:
                    df_master = df
                    flag = True
                else:
                    df_master = pd.concat([df_master, df])
    return df_master

csse_covid_19_daily_reports_us = init_dataframe_from_zip(r'**csv_files/csse_covid_19_daily_reports_us.zip', 'UID')
display(csse_covid_19_daily_reports_us)


,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,ISO3,Testing_Rate,Hospitalization_Rate
UID,,,,,,,,,,,,,,,,,
84000001.0,Alabama,US,2022-01-03 02:33:49,32.3182,-86.9023,909541,16455,NaN,NaN,1.0,18550.003722,6562722.0,NaN,1.809154,USA,133846.102074,NaN
84000002.0,Alaska,US,2022-01-03 02:33:49,61.3707,-152.4044,157169,978,NaN,NaN,2.0,21484.529318,3685702.0,NaN,0.622260,USA,503824.371706,NaN
16.0,American Samoa,US,2022-01-03 02:33:49,-14.2710,-170.1320,10,0,NaN,NaN,60.0,17.972359,2140.0,NaN,0.000000,ASM,3846.084722,NaN
84000004.0,Arizona,US,2022-01-03 02:33:49,33.7298,-111.4312,1390409,24355,NaN,NaN,4.0,19102.391259,15679982.0,NaN,1.751643,USA,215422.333359,NaN
84000005.0,Arkansas,US,2022-01-03 02:33:49,34.9697,-92.3731,572822,9196,NaN,NaN,5.0,18981.418276,4441472.0,NaN,1.605385,USA,147175.628371,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84000051.0,Virginia,US,2021-01-18 05:30:33,37.7693,-78.1700,439305,5729,35276.0,398300.0,51.0,5146.787208,4807396.0,NaN,1.304105,USA,56322.245900,NaN
84000053.0,Washington,US,2021-01-18 05:30:33,47.4009,-121.4905,289939,3903,NaN,NaN,53.0,3807.525595,4179833.0,NaN,1.346145,USA,54890.239429,NaN
84000054.0,West Virginia,US,2021-01-18 05:30:33,38.4912,-80.9545,108821,1776,80187.0,26858.0,54.0,6072.102344,1767500.0,NaN,1.632038,USA,98624.722191,NaN
